In [ ]:
# обрезать до нижней половины
def cut_in_half(image):
    
    height, width, channels = image.shape
    hei = int(height / 2)
    cut_img = image[hei: height, 0:width]
    
    return cut_img

# (НЕ ПОДХОДИТ, но вполне можно переписать) преобразование синих пикселей в белые
def change_blue_pixels_to_white(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([125, 50, 50], dtype=np.uint8)  # Initial lower blue range
    upper_blue = np.array([130, 255, 255], dtype=np.uint8)  # Initial upper blue range

    # Считаем пороги в зависимости от содержимого фото
    blue_mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
    blue_pixels = cv2.countNonZero(blue_mask)
    while blue_pixels == 0:
        # Если не нашли синих пикселей, то расширяем пороги
        blue_histogram = cv2.calcHist([hsv_image], [0], blue_mask, [180], [0, 180])
        peak_index = np.argmax(blue_histogram)
        lower_blue = np.array([peak_index-5, 50, 50], dtype=np.uint8)
        upper_blue = np.array([peak_index+5, 255, 255], dtype=np.uint8)

    mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
    changed_image = image.copy()
    changed_image[mask > 0] = (255, 255, 255)

    return changed_image
    
def enhance_image(image):

    # Блюр артефактов
    denoised = cv2.bilateralFilter(image, 15, 75, 75)

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Повышаем контрастность
    enhanced_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,15,23)

    return enhanced_image

# для удобства, чтобы большие картинки в юпитере отображать
def cv2_to_pil(image):
    
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)
    
    return pil_image

# основная функция преобразования изображения
def preprocess_image(image):
    
    temp = cut_in_half(image)
#    temp = change_blue_pixels_to_white(temp)
    temp = enhance_image(temp)
    final_image = cv2_to_pil(temp)
    
    return final_image

input_dir = "./datasets/405/"
preprocessed_dir = "./datasets/405_preprocessed/"

for im in pairs_list:
    img = cv2.imread(input_dir+im+'.jpg')
    result = preprocess_image(img)
    display(result)
    result = result.save(preprocessed_dir+im+'.jpg')